# Tworzenie datasetu do wykorzystania dla modelu

In [ ]:
import numpy as np
import pandas as pd
from grounding_dino.groundingdino.util.inference import load_image
import cv2

Pobranie odpowiednich danych

In [10]:
masks_array = np.load('data2/masks_array.npy')
incomplete_detect = np.load('data2/incomplete_detect.npy')
df = pd.read_csv('data2/df.csv')

Znalezienie środków dla każdej maski/elementu chwytaka

In [13]:
xyz_array = []

for i in range(0, masks_array.shape[0]):

  # Załadowanie obrazu głębokości
  depth_img, _ = load_image(df['depth_path'][i])
  depth_img = depth_img[:, :, 0]

  tmp = []

  for j in range(0, masks_array.shape[1]):

    # Utworzenie maski
    mask = (masks_array[i][j] * depth_img)
  
    # Obliczanie momentów
    moments = cv2.moments(mask)

    # Wysokość maski
    y_indices, x_indices = np.where(mask > 0)
    if len(y_indices) == 0 or len(x_indices) == 0:
      tmp.extend([0, 0, 0,])
      continue
    mask_height = np.max(y_indices) - np.min(y_indices) + 1
    mask_width = np.max(x_indices) - np.min(x_indices) + 1

    # Momenty centralne
    m10 = moments['m10']
    m01 = moments['m01']
    m00 = moments['m00']

    # Środki masy
    if m00 == 0:
      tmp.extend([0, 0, 0,])
    else:
      x = m10/m00
      y = m01/m00
      z = np.mean(mask[mask > 0])
      tmp.extend([x, y, z])
  xyz_array.append(tmp)

xyz_array = np.array(xyz_array)
print(f"xyz_array shape -> {xyz_array.shape}")

xyz_array shape -> (172, 9)


In [14]:
model_cols = ['ESJoint1',	'ESJoint2',	'ESJoint3',	'ESJoint4',	'ESJoint5',	'ESJoint6',
              'gripper_finger_1_joint',	'gripper_finger_2_joint']
df_array = df[model_cols].to_numpy()
df_array.shape

(172, 8)

połączenie całego przetworzonego zbioru danych

In [15]:
data = np.concatenate((xyz_array, df_array), axis=1)
data.shape

(172, 17)

Usunięcie danych, gdzie nie wykryto maski

In [16]:
# delete incomplete_detect
data = np.delete(data, incomplete_detect, axis=0)
data.shape

(130, 17)

Utworzenie zbioru danych do uczenia - pary pozycji

In [ ]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

k = 62
robot_coords = data[:, 9:17]
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(robot_coords)

ds = []
for i in tqdm(range(data.shape[0])):
    distances, indices = nbrs.kneighbors([robot_coords[i]])
    # indices[0][1:] - bo pierwszy to te same współrzędne
    for j in indices[0][1:]:
        ds.append(np.concatenate([
            data[i, 0:9] - data[j, 0:9],  # X - przemiezczenie elementów chwytaka
            data[i, 9:17],                # X - współrzędne robota 1
            data[i, 9:17] - data[j, 9:17] # Y - różnica współrzędnych 
        ]))
print(len(ds))
ds = np.array(ds)
print(ds.shape)
print(ds[0])

y_cols =[
  'x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3',
  'J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'G1', 'G2',
  'dJ1', 'dJ2', 'dJ3', 'dJ4', 'dJ5', 'dJ6', 'dG1', 'dG2'
]
df_training = pd.DataFrame(ds, columns=y_cols)
df_training.to_csv('data2/df_training.csv', index=False)

100%|██████████| 130/130 [00:00<00:00, 1442.96it/s]


8060
(8060, 25)
[ 8.87935516e-01 -5.47134821e-01 -1.71423340e+00  2.45226331e-01
  1.68362086e-01 -9.46197510e-02  6.87337319e-02 -3.65825385e-01
 -4.08859253e-01  3.42310500e+00  1.23096900e+00  1.93243200e+00
  1.52780100e+00  1.58762000e+00  2.66337400e+00  6.03365000e-01
  9.05047000e-01 -2.94000000e-04  5.88000000e-04  2.94000000e-04
 -5.27000000e-04  1.72600000e-03 -2.20500000e-03 -5.11300000e-03
  0.00000000e+00]
